### Import block
I still need to figure out how to get things to auto-reload, but that's a future project.

In [ ]:
# Import section
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mdwtools import mdwfunctions as mwfn
from mdwtools import mdwplots as mwp

import cesm1to2plotter as c1to2p

import os

### Load Section
This section is where we set all of the flags and load options

In [ ]:
# Set flags/options section
diff_flag = False
loadErai_flag = False  # True to load ERAI fields
loadGpcp_flag = True
loadHadIsst_flag = False
mp_flag = False  # True to use multiprocessing when regridding
newRuns_flag = False
obs_flag = False
ocnOnly_flag = True  # Need to implement to confirm CTindex is right.
regridVertical_flag = False
regrid2file_flag = True
regridOverwrite_flag = True
reload_flag = False
save_flag = False
saveDir = c1to2p.setfilepaths()[2]
saveSubDir = ''  # 'testfigs/66to125/'
saveThenClose_flag = True
verbose_flag = False

fns_flag = True
fnt_flag = True
lts_flag = False
prect_flag = True

#### This is where we actually do the loading.

In [ ]:
# Load data section
# Set name(s) of file(s) to load
versionIds = ['01',
              '28',
              '36',
              'ga7.66',
              # '100',
              # '113',
              # '114',
              # '116',
              # '118',
              '119',
              # '119f',
              # '119f_gamma',
              # '119f_ice',
              # '119f_liqss',
              # '119f_microp',
              # '119f_pra',
              '125',
              # '125f',
              '161',
              '194',
              '195'
              # '297f',
              # '297f_microp',
              # '297f_sp',
              # '297f_pra',
              ]

# Set levels for vertically regridding
newLevs = np.array([100, 200, 275, 350, 425,
                    500, 550, 600, 650, 700,
                    750, 800, 850, 900, 950,
                    975, 1000
                    ])

# Set variables to regrid vertically
#   Number of variables I can load is very memory dependent.
regridVars = ['V', 'OMEGA', 'RELHUM', 'CLOUD', 'T', 'U',
              'AREI', 'AREL', 'AWNC', 'AWNI',
              'CLDICE', 'CLDLIQ',
              # 'ICIMR', 'ICWMR',
              ]

# Get directory of file to load
ncDir, ncSubDir, saveDir2 = c1to2p.setfilepaths(
    loadClimo_flag=True,
    newRuns_flag=newRuns_flag,
    )
if saveDir is None:
    saveDir = saveDir2

# Determine if need to load new datasets
try:
    if not all([vid in dataSets.keys() for vid in versionIds]):
        load_flag = True
    else:
        if regridVertical_flag:
            # Ensure that vertical regrid dataset exists
            try:
                # Esnure all requested regridded variables are present
                if not all([regridVar in dataSets_rg[versionIds[0]]
                            for regridVar in regridVars]):
                    load_flag = True
                else:
                    load_flag = False
            except NameError:
                load_flag = True
        else:
            load_flag = False
except NameError:
    load_flag = True
    
# Load (or reload) datasets from file and regrid if requested/needed
if load_flag:
    dataSets, dataSets_rg = c1to2p.loadmodelruns(
        versionIds,
        loadClimo_flag=True,
        lts_flag=lts_flag,
        mp_flag=mp_flag,
        ncDir=ncDir,
        ncSubDir=ncSubDir,
        newLevs=newLevs,
        regrid2file_flag=regrid2file_flag,
        regridOverwrite_flag=regridOverwrite_flag,
        regridVars=regridVars,
        regridVertical_flag=regridVertical_flag,
        )
print('----------------\n| Done loading |\n----------------')

In [ ]:
dataSets.keys()

### Plotting section
#### Plot multiple maps on one figure for easy comparison

In [ ]:
save_flag = False

# Set plotting options
plotIdList = [  # '125f', '119f_microp', '119f_gamma',
              # '119f_pra', '119f_ice', '119f_liqss',
              # '297f', None, None,
              '01', '28', '36',
              'ga7.66', '119', '125',
              '161', '194', '195',
              ]
plotVars = ['SWCF'] # , 'CLDTOT', 'FNS']  # , 'T', 'RELHUM', 'CLOUD']
#        'CDNUMC']  # , 'SWCF', 'PBLH']
plevs = [1000]
box_flag = False
boxLat = np.array([-30, 30])
boxLon = np.array([240, 270])
diff_flag = True
diffDs = None  # dataSets_rg
diffIdList = ['01']*9
              # ['119f', '119f', '119f',
              # '119f', '119f', '119f',
              # '119f', None, None,
              # '119', '119f', None,
              # '119f', '119f', '119f',
              # ]
diffVar = None
diffPlevs = plevs

quiver_flag = True
uVar = 'TAUX'
vVar = 'TAUY'
levels = None  # np.arange(-1., 1.01, 0.1)

tSteps = np.arange(0, 12)

for plotVar in plotVars:
    for jlev, plev in (enumerate(plevs) if plotVar == 'T' else enumerate([1000])):
        if diffVar is None:
            diffVar = plotVar
        c1to2p.plotmultilatlon((dataSets_rg
                                if plotVar
                                in dataSets_rg[plotIdList[0]]
                                else dataSets),
                               plotIdList,
                               plotVar,
                               box_flag=box_flag,
                               boxLat=boxLat,
                               boxLon=boxLon,
                               cbar_flag=True,
                               cbarOrientation='vertical',
                               compcont_flag=False,
                               diff_flag=diff_flag,
                               diffIdList=diffIdList,
                               diffDs=diffDs,
                               diffPlev=diffPlevs[jlev],
                               diffVar=diffVar,
                               figSize=[18, 6],
                               fontSize=24,
                               latLim=np.array([-30.1, 30.1]),
                               latlbls=np.arange(-30, 30.1, 10),
                               levels=levels,
                               lonLim=np.array([99.5, 290.5]),
                               lonlbls=np.arange(120, 270.1, 30),
                               obsDs=None,  # gpcpClimoDs,
                               ocnOnly_flag=False,
                               plev=plev,
                               quiver_flag=quiver_flag,
                               quiverNorm_flag=False,
                               quiverScale=(
                                   c1to2p.getquiverprops(
                                        uVar, vVar,
                                        diff_flag=diff_flag)['quiverScale']
                                   ),
                               quiverUnits='inches',
                               rmRegLatLim=np.array([-20, 20]),
                               rmRegLonLim=np.array([119.5, 270.5]),
                               rmRegMean_flag=False,
                               rmse_flag=False,
                               save_flag=save_flag,
                               saveDir=(saveDir +
                                        saveSubDir +
                                        'atm/maps/'
                                        ),
                               stampDate_flag=False,
                               subFigCountStart='a',
                               subSamp=7,
                               tSteps=tSteps,
                               uRef=c1to2p.getquiverprops(
                                   uVar, vVar,
                                   diff_flag=diff_flag)['uRef'],
                               uVar=uVar,
                               vVar=vVar,
                               verbose_flag=False,
                               )
        if diffVar == plotVar:
            diffVar = None
print('done')
plt.show()

#### Plot multiple pressure-latitude maps on one figure for ease of comparison

In [ ]:
saveDir

In [ ]:
save_flag = False

# Set variable to plot with colored contours
colorVars = ['CLDICE', 'CLDLIQ']  # , 'AREL', 'CLDLIQ', 'CLDICE',
#             'T', 'RELHUM', 'CLOUD', 'OMEGA']
plotCases = [  # '125f', None, None,
             '125f', '119f_microp', '119f_gamma',
              '119f_pra', '119f_ice', '119f_liqss',
              '297f', None, None
             # '125', '125f', None,
             # '119f_microp', '119f_gamma', '119f_liqss'
             ]
diff_flag = True
lineCont_flag = False
lineContDiff_flag = False

diffIdList = ['119f', '119f', '119f',
              '119f', '119f', '119f',
              '119f', None, None,
              ]
diffCase = {'119f': '119f_microp',
            '119f_gamma': '119f',
            '119f_liqss': '119f',
            '119f_microp': '119f',
            '125': '119',
            '125f': '119f',
            'cesm20f': '119f',
            'cesm20f_microp': 'cesm20f',
            'cesm20f_sp': '119f',
            }

# Set variable to plot with black contours
contVar = None  # 'CLOUD'

quiver_flag = True
# save_flag = False

# Set plotting limits
latLim = np.array([-40, 40])
lonLim = np.array([240, 270])
pLim = np.array([1000, 200])
tLim = np.array([0, 12])  # exclusive of end pt.
dt = 1

# Loop through variables and plot them.
for colorVar in colorVars:
    c1to2p.plotmultipressurelat(
        dataSets_rg,
        plotCases,
        colorVar,
        # caseString=None,
        cbar_flag=True,
        # cbar_dy=-0.1,
        # cbar_height=0.02,
        # cMap=None,
        colorConts=None,  # np.arange(-5, 5.01, 0.5),
        # dCont_flag=False,
        # dContCase=None,
        diff_flag=diff_flag,
        diffIdList=([None if vid is None
                     else diffCase[vid]
                     for vid in plotCases]
                     if diffIdList is None
                     else diffIdList),
        dt=1,
        latLbls=np.arange(-30, 30.1, 10),
        latLim=latLim,
        latSubSamp=3,
        lonLim=lonLim,
        lineCont_flag=lineCont_flag,
        # lineContDiff_flag=lineContDiff_flag,
        # lineConts=None,
        # lineContVar=colorVar,
        # lineContDs=dataSets_rg,
        # lineContDiffIdList=None,
        pLim=pLim,
        quiver_flag=quiver_flag,
        # quiverScale=3,
        # quiverUnits='inches',
        save_flag=save_flag,
        saveDir=saveDir,
        saveSubDir='',  # atm/meridslices/',
        saveThenClose_flag=saveThenClose_flag,
        tLim=tLim,
        wScale=100,
        )
print('done')
plt.show()